In [ ]:
!pip install igraph
!pip install scikit-image==0.20.0

In [1]:
from GNEMS.GNEMS import GNEMS_segment
from PIL import Image
from PIL.PngImagePlugin import PngInfo
import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
from skimage.color import label2rgb
from glob import glob
from tqdm import tqdm
import time
import threading
import json
import os

/Users/isaac/miniforge3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
def run_tests(hyperparams, label=None, max_threads=8):
    if label is None:
        method_name = f"GNEMS_alternate_{time.time()}"
    else:
        method_name = f"GNEMS_{label}"

    assert not os.path.exists(f"results/{method_name}"), "Directory already exists!"

    !mkdir results/{method_name}
    !mkdir results/{method_name}/noise
    !mkdir results/{method_name}/noise/0.5
    !mkdir results/{method_name}/noise/1.0
    !mkdir results/{method_name}/noise/2.0
    !mkdir results/{method_name}/noise/4.0
    !mkdir results/{method_name}/noise/8.0
    !mkdir results/{method_name}/clouds
    !mkdir results/{method_name}/texture

    with open(f"results/{method_name}/hyperparams.json", "w") as f:
        json.dump(hyperparams, f)

    def segment_and_save(path, id, bar=None):
        im = np.array(Image.open(path).convert("L"))
        d = hyperparams["d"]
        iterations = hyperparams["iterations"]
        prediction_stride = hyperparams["prediction_stride"]
        lambda_ = hyperparams["lambda_"]
        slic_segments = hyperparams["slic_segments"]
        sigma = hyperparams["sigma"]
        seg = GNEMS_segment(im, d=d, lambda_=lambda_, iterations=iterations, prediction_stride=prediction_stride, slic_segments=slic_segments, sigma=sigma, show_progress=False)
        category = path.split("/")[1]
        Image.fromarray(seg).convert("L").save(f"results/{method_name}/{category}/{id}.png")
        bar.update(1)

    print("Segmenting clouds dataset...")
    with open("datasets/clouds/test_ids.txt", "r") as f:
        test_ids = [line.strip() for line in f.readlines()]

    threads = []
    with tqdm(total = len(test_ids)) as pbar:
        for id in test_ids:
            path = f"datasets/clouds/images/{id}.png"
            t = threading.Thread(target=segment_and_save, args=(path,id,pbar))
            t.start()
            threads.append(t)
            while len(threading.enumerate()) >= max_threads:
                time.sleep(1)
        for t in threads:
            t.join()

    with open("datasets/noise/test_ids.txt", "r") as f:
        test_ids = [line.strip() for line in f.readlines()]

    print("Segmenting noise dataset...")
    threads = []
    with tqdm(total = len(test_ids)) as pbar:
        for id in test_ids:
            path = f"datasets/noise/{id}.png"
            t = threading.Thread(target=segment_and_save, args=(path,id,pbar))
            t.start()
            threads.append(t)
            while len(threading.enumerate()) >= max_threads:
                time.sleep(1)
        for t in threads:
            t.join()

    print("Segmenting texture dataset...")
    with open("datasets/texture/test_ids.txt", "r") as f:
        test_ids = [line.strip() for line in f.readlines()]

    threads = []
    with tqdm(total = len(test_ids)) as pbar:
        for id in test_ids:
            path = f"datasets/texture/images/{id}.png"
            t = threading.Thread(target=segment_and_save, args=(path,id,pbar))
            t.start()
            threads.append(t)
            while len(threading.enumerate()) >= max_threads:
                time.sleep(1)
        for t in threads:
            t.join()

In [10]:
hyperparams = {
    "d": 16,
    "iterations": 50,
    "prediction_stride": 4,
    "lambda_": 0.3,
    "n_filters": 16,
    "slic_segments": 100,
    "sigma": 3
}
run_tests(hyperparams, label="lower_stride", max_threads=1)

Segmenting clouds dataset...


  0%|          | 1/256 [00:47<3:20:40, 47.22s/it]


KeyboardInterrupt: 